In [40]:
import numpy as np
import numpy.random as rd
import pandas as pd
from IPython.display import display, Math, HTML

In [41]:
def f(u):
    return np.cos(2 * np.pi * (np.mean(u) - 1/2))

Notice that $I := \int_{[0, 1]^d} f(u) du = \mathbb{E}_{U \sim \mathcal{U}([0, 1]^d)}[f(U)]$. Thus, we can approximate $I$ using Monte Carlo.

In [43]:
def montecarlo(d, N = 10000):
    X = [f(rd.random(d)) for _ in range(N)]
    return np.mean(X)

In [44]:
# Results for different values of d and N

d_values = [1, 2, 3, 5, 10, 100]
N_values = [100, 1000, 10000]

data = {N: [montecarlo(d, N) for d in d_values] for N in N_values}

df = pd.DataFrame(data, index = d_values)
df.iloc[:len(d_values)]

,100,1000,10000
1,0.011286,0.000376,-0.009747
2,0.359022,0.408734,0.405004
3,0.564058,0.581170,0.566068
5,0.709669,0.706741,0.709269
10,0.850946,0.851896,0.846715
100,0.986757,0.982488,0.983756


In [46]:
def quasimontecarlo(d = 1, N = 10000):
    if d == 1:
        X = [f((2 * n - 1)/(2 * N)) for n in range(1, N + 1)]
        return np.mean(X)

# Cas d > 1 à traiter

In [47]:
montecarlo(1), quasimontecarlo(1)

# Quasi-MC est beaucoup mieux que MC

(np.float64(-0.015555535832659242), np.float64(4.5474735088646414e-17))